In [579]:
using QuantumOptics

In [611]:
b = SpinBasis(1//2)
sx = sigmax(b)
sy = sigmay(b)
sz = sigmaz(b)

Operator(dim=2x2)
  basis: Spin(1/2)sparse([1, 2], [1, 2], ComplexF64[1.0 + 0.0im, -1.0 + 0.0im], 2, 2)

In [627]:
factor = 7
Bmag = 10

10

In [628]:
function B0(t)
    return Bmag
end

function θ(t)
    return pi/factor
end

function ϕ(t)
    return t
end

function Bx(t)
    return B0(t)*sin(θ(t))*cos(ϕ(t))
end

function By(t)
    return B0(t)*sin(θ(t))*sin(ϕ(t))
end

function Bz(t)
    return B0(t)*cos(θ(t))
end

Bz (generic function with 1 method)

In [629]:
Ht = TimeDependentSum(Bx=>sx, By=>sy, Bz=>sz)

TimeDependentSum(dim=2x2)
  basis: Spin(1/2)

In [630]:
t=0
E0, Psi0 = eigenstates(dense(Bx(t)*sx+By(t)*sy+Bz(t)*sz));

In [642]:
ψ₀ = Psi0[1]
dt = 0.01
tspan = [0:dt:2*pi+dt;];

In [643]:
@time begin
tout, ψₜ = timeevolution.schroedinger_dynamic(tspan, ψ₀, Ht);
end;

  0.002702 seconds (39.32 k allocations: 1.681 MiB)


In [644]:
H = LazySum([0.0, 0.0, 0.0],[sx,sy,sz])
function H_B(t, psi)
  H.factors[1] = Bx(t)
  H.factors[2] = By(t)
  H.factors[3] = Bz(t)
  return H
end

H_B (generic function with 1 method)

In [645]:
@time begin
tout2, ψₜ2 = timeevolution.schroedinger_dynamic(tspan, ψ₀, H_B);
end;

  0.021338 seconds (40.78 k allocations: 1.919 MiB, 89.11% compilation time: 60% of which was recompilation)


In [646]:
t=0 
eigenenergies(dense(Bx(t)*sx+By(t)*sy+Bz(t)*sz))

2-element Vector{Float64}:
 -10.000000000000002
  10.000000000000002

In [647]:
Elist=[]
for t in tspan
    Evals = eigenenergies(dense(Bx(t)*sx+By(t)*sy+Bz(t)*sz))
    push!(Elist,Evals[1])
end

In [648]:
#import Pkg
#Pkg.add("NumericalIntegration")

In [649]:
using NumericalIntegration

In [650]:
dynamic_phase = - NumericalIntegration.integrate(tspan,Elist)
dynamic_phase/(2*pi), angle( dagger(ψₜ[end])*ψ₀ )/pi, angle( dagger(ψₜ2[end])*ψ₀ )/pi

(10.001296623895442, -0.10652820022039625, -0.10652820022039625)

In [651]:
solid_angle = 2*pi*(1-cos(pi/factor))

0.6222309541488807

In [652]:
mod( angle( dagger(ψ₀)*ψₜ[end] ) - dynamic_phase, 2*pi)/pi, solid_angle/2/pi

(0.10393495242951117, 0.09903113209758085)